In [1]:
import os
import shutil

# Get user profile path
user_profile = os.environ["USERPROFILE"]

# Paths to Hugging Face cached models
cached_models = [
    os.path.join(user_profile, r".cache\huggingface\hub\models--bert-base-multilingual-cased"),
    os.path.join(user_profile, r".cache\huggingface\hub\models--xlm-roberta-base")
]

# Remove cached models if they exist
for path in cached_models:
    if os.path.exists(path):
        shutil.rmtree(path)
        print(f"Removed cache: {path}")
    else:
        print(f"No cache found at: {path}")


Removed cache: C:\Users\stdFurqan\.cache\huggingface\hub\models--bert-base-multilingual-cased
Removed cache: C:\Users\stdFurqan\.cache\huggingface\hub\models--xlm-roberta-base


In [2]:
# Set device
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
# ==============================
# Imports
# ==============================
import os
import re
import torch
import pandas as pd
import random
import numpy as np
from collections import Counter
from transformers import XLMRobertaTokenizer, XLMRobertaForMaskedLM
from openprompt.prompts import ManualTemplate, ManualVerbalizer
from openprompt.data_utils import InputExample
from openprompt.plms import load_plm
from openprompt import PromptForClassification, PromptDataLoader
from torch.optim import AdamW
from sklearn.metrics import classification_report
from collections import defaultdict
from torch.utils.data import DataLoader, Sampler

# ========================================
# Check CUDA
# ========================================
device = "cuda" #if torch.cuda.is_available() else "cpu"
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU name:", torch.cuda.get_device_name(0))
    print("CUDA version:", torch.version.cuda)
    print("GPU count:", torch.cuda.device_count())

# ========================================
# Seeds for reproducibility
# ========================================
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

C:\Users\stdFurqan\anaconda3\envs\py310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


CUDA available: True
GPU name: NVIDIA GeForce RTX 4080 SUPER
CUDA version: 12.1
GPU count: 1


In [4]:
### Classes ###
classes = ['P', 'NP']

### Label Map ###
label_map = {'P': 1, 'NP': 0}



# # Step 1: Use load_plm with 'roberta' to get the correct WrapperClass
_, _, _, WrapperClass = load_plm("roberta", "roberta-base")  # Just to get the wrapper

# # # Step 2: Manually load XLM-RoBERTa model/tokenizer
model_name = "xlm-roberta-base"
tokenizer = XLMRobertaTokenizer.from_pretrained(model_name)
plm = XLMRobertaForMaskedLM.from_pretrained(model_name)

# ==============================
# Load Pretrained Language Model (mBERT)
# ==============================
# plm, tokenizer, model_config, WrapperClass = load_plm("bert", "bert-base-multilingual-cased")

C:\Users\stdFurqan\anaconda3\envs\py310\lib\site-packages\huggingface_hub\file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
C:\Users\stdFurqan\anaconda3\envs\py310\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\stdFurqan\.cache\huggingface\hub\models--xlm-roberta-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as

In [5]:
template = ManualTemplate(
    text = '{"placeholder":"text_a"} اور {"placeholder":"text_b"} کیا ایک جیسے ہیں؟ {"mask"}',
    tokenizer=tokenizer,
)






# templates = [
#     ("T1", ManualTemplate(
#         text='{"placeholder":"text_a"} اور {"placeholder":"text_b"} کیا ایک جیسے ہیں؟ {"mask"}',
#         tokenizer=tokenizer,
#     )),

#     ("T2", ManualTemplate(
#         text='{"placeholder":"text_a"} اور {"placeholder":"text_b"} میں معنی کی مطابقت {"mask"} ہے۔',
#         tokenizer=tokenizer,
#     )),

#     ("T3", ManualTemplate(
#         text='کیا یہ دونوں جملے {"placeholder":"text_a"} اور {"placeholder":"text_b"} ایک دوسرے کی تکرار ہیں؟ {"mask"}',
#         tokenizer=tokenizer,
#     )),

#     ("T4", ManualTemplate(
#         text='{"placeholder":"text_a"} کے مطابق {"placeholder":"text_b"} {"mask"} ہے۔',
#         tokenizer=tokenizer,
#     )),

#     ("T5", ManualTemplate(
#         text='دونوں جملے {"placeholder":"text_a"} اور {"placeholder":"text_b"} کا تعلق {"mask"} ہے۔',
#         tokenizer=tokenizer,
#     )),

#     ("T6", ManualTemplate(
#         text='{"placeholder":"text_b"} اور {"placeholder":"text_a"} ایک جیسے خیال رکھتے ہیں؟ {"mask"}',
#         tokenizer=tokenizer,
#     )),

#     ("T7", ManualTemplate(
#         text='{"placeholder":"text_a"} اور {"placeholder":"text_b"} کے الفاظ کی ہم آہنگی {"mask"} ہے۔',
#         tokenizer=tokenizer,
#     )),

#     ("T8", ManualTemplate(
#         text='اگر ہم {"placeholder":"text_a"} اور {"placeholder":"text_b"} کو دیکھیں تو یہ {"mask"} ہیں۔',
#         tokenizer=tokenizer,
#     )),

#     ("T9", ManualTemplate(
#         text='{"placeholder":"text_a"} کی روشنی میں {"placeholder":"text_b"} {"mask"} سمجھا جا سکتا ہے۔',
#         tokenizer=tokenizer,
#     )),

#     ("T10", ManualTemplate(
#         text='{"placeholder":"text_a"} اور {"placeholder":"text_b"} کے بیچ مفہوم کا تعلق {"mask"} ہے۔',
#         tokenizer=tokenizer,
#     )),
# ]










verbalizer = ManualVerbalizer(
    classes=classes,
    label_words={
        "P": ["ہم معنی", "ایک جیسے"],    # Paraphrase: same meaning / similar
        "NP": ["مختلف", "غیر مترادف"] 
    },
    tokenizer=tokenizer,
)

In [6]:
# ==============================
# Create Prompt Model
# ==============================
prompt_model = PromptForClassification(
    template=template,
    plm=plm,
    verbalizer=verbalizer
)


In [7]:
# ==============================
# Helper Function to Read XML
# ==============================
def read_xml_text(path):
    """
    Reads an XML file and returns cleaned text inside <UPPC_document>...</UPPC_document>.
    Removes all tags and converts inner double quotes to single quotes.
    """
    with open(path, "r", encoding="utf-8") as f:
        xml = f.read()
    
    # Extract text between opening and closing <UPPC_document>
    start = xml.find(">") + 1
    end = xml.rfind("</UPPC_document>")
    text = xml[start:end].strip()
    
    # Remove any remaining tags inside the text
    text = re.sub(r"<.*?>", "", text)
    
    # Replace multiple spaces/newlines with single space
    text = re.sub(r"\s+", " ", text).strip()
    
    # Replace double quotes inside text with single quotes
    text = text.replace('"', "'")
    
    return text

# ==============================
# Load Test CSV
# ==============================
df = pd.read_csv(r"C:\Users\stdFurqan\Desktop\paft\PARA\test_pairs.csv")

data_folder = r"C:\Users\stdFurqan\Desktop\paft\PARA\data"

# ==============================
# Make InputExamples
# ==============================
eval_dataset = []
for i, row in df.iterrows():
    # Read XML texts
    text_a_path = os.path.join(data_folder, row['source_file'])
    text_b_path = os.path.join(data_folder, row['target_file'])
    
    text_a = read_xml_text(text_a_path)
    text_b = read_xml_text(text_b_path)
    
    eval_dataset.append(
        InputExample(
            guid=i,
            text_a=text_a,
            text_b=text_b,
            label=label_map[row['label']]
        )
    )


# ==============================
# PromptDataLoader
# ==============================
eval_loader = PromptDataLoader(
    dataset=eval_dataset,
    tokenizer=tokenizer,
    template=template,              # your ManualTemplate for XNLI
    tokenizer_wrapper_class=WrapperClass,
    max_seq_length=128,
    batch_size=8,
    shuffle=False
)

# ==============================
# Evaluate Model
# ==============================
prompt_model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for batch in eval_loader:
        logits = prompt_model(batch)
        preds = torch.argmax(logits, dim=-1)
        all_preds.extend(preds.cpu().tolist())
        all_labels.extend(batch['label'].cpu().tolist())

# ==============================
# Print Classification Report
# ==============================
print("\n📊 PARA Urdu Dev Classification Report:")
print(classification_report(all_labels, all_preds, target_names=classes, digits=4))

tokenizing: 0it [00:00, ?it/s]Token indices sequence length is longer than the specified maximum sequence length for this model (623 > 512). Running this sequence through the model will result in indexing errors
tokenizing: 28it [00:00, 664.12it/s]



📊 PARA Urdu Dev Classification Report:
              precision    recall  f1-score   support

           P     0.4643    1.0000    0.6341        13
          NP     0.0000    0.0000    0.0000        15

    accuracy                         0.4643        28
   macro avg     0.2321    0.5000    0.3171        28
weighted avg     0.2156    0.4643    0.2944        28



C:\Users\stdFurqan\anaconda3\envs\py310\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\stdFurqan\anaconda3\envs\py310\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\stdFurqan\anaconda3\envs\py310\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(resu

In [ ]:
roberta
📊 PARA Urdu Dev Classification Report:
              precision    recall  f1-score   support

           P     0.4643    1.0000    0.6341        13
          NP     0.0000    0.0000    0.0000        15

    accuracy                         0.4643        28
   macro avg     0.2321    0.5000    0.3171        28
weighted avg     0.2156    0.4643    0.2944        28

In [ ]:
mbert
📊 PARA Urdu Dev Classification Report:
              precision    recall  f1-score   support

           P     0.4643    1.0000    0.6341        13
          NP     0.0000    0.0000    0.0000        15

    accuracy                         0.4643        28
   macro avg     0.2321    0.5000    0.3171        28
weighted avg     0.2156    0.4643    0.2944        28

In [ ]:
# # ==============================
# # Load Evaluation Dataset
# # ==============================
# df = pd.read_csv(r"C:\Users\stdFurqan\Desktop\paft\SST-2\urdu_sentiment_test_labeled.csv")
# eval_dataset = [
#     InputExample(guid=i, text_a=row['text'], label=label_map[row['label']])
#     for i, row in df.iterrows()
# ]

# # ==============================
# # 0-Shot Evaluation with Each Template
# # ==============================
# prompt_model.eval()  # ensure model is in evaluation mode
# batch_size = 8    # eval batch size

# # Optional: store template order and results
# all_pass_patterns = {}

# for pass_idx, (prompt_name, current_template) in enumerate(templates, start=1):
#     print(f"\n🟦 0-Shot Evaluation - Template {prompt_name} ({pass_idx}/{len(templates)})")

#     # Create PromptDataLoader with current template
#     eval_loader = PromptDataLoader(
#         dataset=eval_dataset,
#         tokenizer=tokenizer,
#         template=current_template,
#         tokenizer_wrapper_class=WrapperClass,
#         max_seq_length=128,
#         batch_size=batch_size,
#         shuffle=False
#     )

#     pass_preds = []
#     pass_labels = []

#     # Run evaluation
#     with torch.no_grad():
#         for batch in eval_loader:
#             logits = prompt_model(batch)
#             preds = torch.argmax(logits, dim=-1)
#             pass_preds.extend(preds.cpu().tolist())
#             pass_labels.extend(batch['label'].cpu().tolist())

#     # Print report immediately after this template
#     print(f"\n📊 STS_B Urdu Dev Classification Report - Template {prompt_name}")
#     print(classification_report(pass_labels, pass_preds, target_names=classes, digits=4))

#     # Store template name (optional)
#     all_pass_patterns[f"pass_{pass_idx}"] = prompt_name

# # Optional: print template order at the end
# print("\n✅ Templates used per pass:", all_pass_patterns)
